# Aerospike Java Client – Reading and Updating Maps

This notebook demonstrates Java Aerospike CRUD operations (Create, Read, Update, Delete) for maps of data, focusing on server-side **read** and **update** operations. 

Aerospike stores records by association with a **key**. Maps contain key:value pairs. This notebook makes use of the word **mapkey** to distinguish from a record **key**. 

This [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/examples_index.html) requires the Aerospike database running locally with Java kernel and Aerospike Java Client. To create a Docker container that satisfies the requirements and holds a copy of these notebooks, visit the [Aerospike Notebooks Repo](https://github.com/aerospike-examples/interactive-notebooks).

## Notebook Setup 

Run these first to initialize Jupyter, download the Java Client, and make sure the Aerospike Database is running.

### Import Jupyter Java Integration 

Make it easier to work with Java in Jupyter.

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;

IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Start Aerospike

Ensure Aerospike database is running locally.

In [2]:
%sh asd

### Download the Aerospike Java Client

Ask Maven to download and install the project object model (POM) of the Aerospike Java Client.

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

### Start the Aerospike Java Client and Connect

Create an instance of the Aerospike Java Client, and connect to the demo cluster.

The default cluster location for the Docker container is *localhost* port *3000*. If your cluster is not running on your local machine, modify *localhost* and *3000* to the values for your Aerospike cluster.

In [4]:
import com.aerospike.client.AerospikeClient;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


## CREATING Maps in Aerospike

### Create and Print Map Data

Create a string map representing fish metadata. Create an integer map containing timestamped fish observation locations.

In [20]:
import java.util.ArrayList;
import java.util.Arrays;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

HashMap <String, String> mapFish = new HashMap <String, String>();
mapFish.put("name", "Annette");
mapFish.put("fruit", "Pineapple");
mapFish.put("color", "Aquamarine");
mapFish.put("tree", "Redwood");
System.out.println("Fish map: " + mapFish);


HashMap <Integer, HashMap> mapObs = new HashMap <Integer, HashMap>();
HashMap <String, Integer> mapCoords0 = new HashMap <String, Integer>();
mapCoords0.put("lat", -85);
mapCoords0.put("long", -130);
HashMap <String, Integer> mapCoords1 = new HashMap <String, Integer>();
mapCoords1.put("lat", -25);
mapCoords1.put("long", -50);
HashMap <String, Integer> mapCoords2 = new HashMap <String, Integer>();
mapCoords2.put("lat", 35);
mapCoords2.put("long", 30);

mapObs.put(12345, mapCoords0);
mapObs.put(13456, mapCoords1);
mapObs.put(14567, mapCoords2);
System.out.println("Observations Map:" + mapObs);

Fish map: {color=Aquamarine, fruit=Pineapple, name=Annette, tree=Redwood}
Observations Map:{13456={lat=-25, long=-50}, 14567={lat=35, long=30}, 12345={lat=-85, long=-130}}


### Insert the Maps into Aerospike

Insert one record in Aerospike with **Key** "koi", and **Bin Names** *mapfishbin* and *mapobsbin*. 

#### Create a Key Java Object

A **Key** identifies a specific record in your Aerospike server or cluster. Each key must have a **Namespace** and optionally a **Set** name. 
* A **Namespace** is like a database, in Aerospike.  
* A **Set** is like a database table in Aerospike. 

The namespace *test* is configured on your Aerospike server or cluster. 

For additional information on the [Aerospike Data Model](https://www.aerospike.com/docs/architecture/data-model.html), go [here](https://www.aerospike.com/docs/architecture/data-model.html). 

In [6]:
import com.aerospike.client.Key;

String mapSet = "mapset1";
String mapNamespace = "test";

String theKey = "koi";

Key key = new Key(mapNamespace, mapSet, theKey);
System.out.println("Key created." );

Key created.


#### Create a Bin Java Object for Each Map

A **Bin** is a data field in an Aerospike record.

In [7]:
import com.aerospike.client.Bin;

String mapFishBin = "mapfishbin";
String mapObsBin = "mapobsbin";

Bin bin1 = new Bin(mapFishBin, mapFish);
Bin bin2 = new Bin(mapObsBin, mapObs);

System.out.println( "Created " + bin1 + " and " + bin2 + ".");

Created mapfishbin:{color=Aquamarine, fruit=Pineapple, name=Annette, tree=Redwood} and mapobsbin:{13456={lat=-25, long=-50}, 14567={lat=35, long=30}, 12345={lat=-85, long=-130}}.


#### Create a Write Policy Java Object for Record Insertion 

A **Policy** tells Aerospike the intent of a database operation. 

For more information on [policies](https://www.aerospike.com/docs/guide/policies.html), go [here](https://www.aerospike.com/docs/guide/policies.html).

In [8]:
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;

ClientPolicy clientPolicy = new ClientPolicy();
System.out.println("Created a client policy.");

Created a client policy.


#### Put the Map Data into Aerospike

In [9]:
client.put(clientPolicy.writePolicyDefault, key, bin1, bin2);
System.out.println("Key: " + theKey + ", mapfishbin: " + mapFish + ", mapobsbin: " + mapObs );

Key: koi, mapfishbin: {color=Aquamarine, fruit=Pineapple, name=Annette, tree=Redwood}, mapobsbin: {13456={lat=-25, long=-50}, 14567={lat=35, long=30}, 12345={lat=-85, long=-130}}


## READING Maps and Map Elements From the Server

Now that the maps are in Aerospike, the client can return full or partial maps from **bin** contents. No data is modified by these ops.

### Get the Record

A record can be retrieved using the **key**, **namespace**, and **set** name.

In the output: 
* **gen** is the generation number, the number of record writes. 
* **exp** is the expiration counter for the record.

For more information on [both generation number and expiration](https://www.aerospike.com/docs/guide/FAQ.html), see the [Aerospike FAQ](https://www.aerospike.com/docs/guide/FAQ.html).

In [10]:
import com.aerospike.client.Record;

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
System.out.println(record);

(gen:41),(exp:352069551),(bins:(mapfishbin:{name=Annette, tree=Redwood, color=Aquamarine, fruit=Pineapple}),(mapobsbin:{13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, 14567={lat=35, long=30}}))


### Get String Elements by Mapkey, Rank, and Value

Aerospike provides **MapOperations** to read string mapkeys and values from the Aerospike Server or Cluster. 

The mapFishBin is a map containing string mapkey/value pairs associated with the fish, "Koi".

For more information on [map operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html).

#### Get String by Mapkey

Aerospike provides operations to look up a value by mapkey. As a convenience, the client returns the specified value as the contents of the bin. 

For the list of [return type options](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapReturnType.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapReturnType.html). 

In [13]:
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapReturnType;
// import com.aerospike.client.cdt.CTX;

String mapKeyToFind = "color";

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record colorString = client.operate(null, key, 
    MapOperation.getByKey(mapFishBin, Value.get(mapKeyToFind), MapReturnType.VALUE)
    );

System.out.println("The string map: " + record.getValue(mapFishBin));
System.out.println("The color in string map is: " + colorString.getValue(mapFishBin));

The string map: {name=Annette, tree=Redwood, color=Aquamarine, fruit=Pineapple}
The color in string map is: Aquamarine


#### Get Highest Rank String

Aerospike provides operations to look up a value by rank within the map. 

For information on [list ranking](https://en.wikipedia.org/wiki/List_ranking), go [here](https://en.wikipedia.org/wiki/List_ranking).

In [16]:
Integer highestRank = -1;

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record highestRankString = client.operate(null, key, 
    MapOperation.getByRank(mapFishBin, highestRank, MapReturnType.VALUE)
    );

System.out.println("The string map: " + record.getValue(mapFishBin));
System.out.println("The highest rank string is: " + highestRankString.getValue(mapFishBin));

The string map: {name=Annette, tree=Redwood, color=Aquamarine, fruit=Pineapple}
The highest rank string is: Redwood


#### Get Mapkey By String Value

Aerospike provides operations to look up a mapkey by the associated value. 

In [17]:
String valueToFind = "Pineapple";

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record foundMapKey = client.operate(null, key, 
    MapOperation.getByValue(mapFishBin, Value.get(valueToFind), MapReturnType.KEY)
    );

System.out.println("The string map: " + record.getValue(mapFishBin));
System.out.println("The mapkey associated with " + valueToFind + " is: " + foundMapKey.getValue(mapFishBin));

The string map: {name=Annette, tree=Redwood, color=Aquamarine, fruit=Pineapple}
The mapkey associated with Pineapple is: [fruit]


### Get Integer Data by Size, Index and Key Range

Aerospike provides operations to read integers associated with fish observations.

The mapobsbin is a list of Latitude/Longitude pairs stored by the time of fish observation in seconds from the start of the experiment. The number of seconds, latitude, and longitude are all integers.

#### Get the Number of Observations in the Map

Aerospike provides a size operation, which returns a count of the mapkeys in a map.

In [18]:
Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record sizeString = client.operate(null, key, 
    MapOperation.size(mapObsBin)
    );

System.out.println("The Observation Map: " + record.getValue(mapObsBin));
System.out.println("The number of Observations in the Map: " + sizeString.getValue(mapObsBin));

The Observation Map: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, 14567={lat=35, long=30}}
The number of Observations in the Map: 3


#### Get The First Observation in the Map 

Maps are sorted by mapkey. **Docs suggest this isn't the default. Is this reliable, or due to my use of a HashMap?** The first element by index represents the first time the fish was observed. Aerospike provides operations to look up a value by index.

Aerospike operations allow indexing forward from the beginning of the list using zero-based numbering. Negative numbers index backwards from the end of a list. 

For examples of [indexes](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html).

In [21]:
Integer firstIdx = 0;

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record firstObservation = client.operate(null, key, 
    MapOperation.getByIndex(mapObsBin, firstIdx, MapReturnType.KEY_VALUE)
    );

System.out.println("The Observation Map: " + record.getValue(mapObsBin));
System.out.println("The First Observation: " + firstObservation.getValue(mapObsBin));

The Observation Map: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, 14567={lat=35, long=30}}
The First Observation: [12345={lat=-85, long=-130}]


#### Get All Locations Observed Between 13,000 and 15,000 seconds.

Aerospike provides operations to get values by mapkey range. Get the latitude and longitude pairs for all observations between 13,000 and 15,000 seconds.  

In [22]:
Integer lowerBound = 13000;
Integer upperBound = 15000;

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record rangeObservations = client.operate(null, key, 
    MapOperation.getByKeyRange(mapObsBin, Value.get(lowerBound), Value.get(upperBound), MapReturnType.KEY_VALUE)
    );

System.out.println("The Observation Map: " + record.getValue(mapObsBin));
System.out.println("The Observations between 13000 and 15000 seconds: " + rangeObservations.getValue(mapObsBin));

The Observation Map: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, 14567={lat=35, long=30}}
The Observations between 13000 and 15000 seconds: [13456={lat=-25, long=-50}, 14567={lat=35, long=30}]


## UPDATING Maps on the Aerospike Server

Aerospike's **MapOperations** can also modify data in the Aerospike database.

### Update the Fish Bin in Aerospike

The Fish Bin contains metadata about the fish.

#### Create a MapPolicy Java Object for the Fish Bin

When modifying maps, Aerospike requires a **MapPolicy** to set the **MapWriteMode** and optionally change the sort order. The mapwritemode determines if the operation should fail if a mapkey or value already exists. 

For more information on [mappolicy](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapPolicy.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapPolicy.html).

For more information on [mapwritemode](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapWriteMode.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapWriteMode.html).

In [23]:
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapWriteMode;

MapPolicy mapFishBinPolicy = new MapPolicy();
System.out.println("Created default MapPolicy for mapFishBin.")

Created default MapPolicy for mapFishBin.


#### Change the Tree to Larch
As a convenience, when new data is put into a map, Aerospike returns the size of the resulting map. 

In [24]:
String treeMapkeyName = "tree";
String newTree = "Larch";

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record sizeOfMapWithNewTree = client.operate(null, key, 
    MapOperation.put(mapFishBinPolicy, mapFishBin, Value.get(treeMapkeyName), Value.get(newTree))
    );
Record mapWithNewTree = client.get(null, key);

System.out.println("The original Fish Map: " + record.getValue(mapFishBin));
System.out.println("The size of the Map after the operation: " + sizeOfMapWithNewTree.getValue(mapFishBin));
System.out.println("The new Fish Map: " + mapWithNewTree.getValue(mapFishBin));

The original Fish Map: {name=Annette, tree=Redwood, color=Aquamarine, fruit=Pineapple}
The size of the Map after the operation: 4
The new Fish Map: {name=Annette, tree=Larch, color=Aquamarine, fruit=Pineapple}


####  Remove the Fruit

As a convenience, when removing a mapkey:value pair, Aerospike client returns the removed data.

In [25]:
String fruitMapkeyName = "fruit";

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record valOfRemovedFruit = client.operate(null, key, 
    MapOperation.removeByKey(mapFishBin, Value.get(fruitMapkeyName), MapReturnType.KEY_VALUE)
    );
Record mapWithoutFruit = client.get(null, key);

System.out.println("The original Fish Map: " + record.getValue(mapFishBin));
System.out.println("The removed mapkey/value pair: " + valOfRemovedFruit.getValue(mapFishBin));
System.out.println("The new Fish Map: " + mapWithoutFruit.getValue(mapFishBin));

The original Fish Map: {name=Annette, tree=Larch, color=Aquamarine, fruit=Pineapple}
The removed mapkey/value pair: [fruit=Pineapple]
The new Fish Map: {color=Aquamarine, name=Annette, tree=Larch}


#### Add Bait

To be sure that other scientists can catch the fish, add the fish's preferrred bait to the record.

In [26]:
String mapkeyForBait = "bait";
String valueForBait = "Mosquito Larva";

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record sizeOfRecordWithBait = client.operate(null, key, 
    MapOperation.put(mapFishBinPolicy, mapFishBin, Value.get(mapkeyForBait), Value.get(valueForBait))
    );
Record recordWithBait = client.get(null, key);

System.out.println("The original Fish Map: " + record.getValue(mapFishBin));
System.out.println("The Fish Map with Bait: " + recordWithBait.getValue(mapFishBin));

The original Fish Map: {color=Aquamarine, name=Annette, tree=Larch}
The Fish Map with Bait: {name=Annette, tree=Larch, color=Aquamarine, bait=Mosquito Larva}


### Update the Observation Map in Aerospike
Aerospike provides operations to update integers and sub-maps, also.

#### Create a MapPolicy Java Object for the Observations Bin

In [27]:
MapPolicy mapObsBinPolicy = new MapPolicy();
System.out.println("Created default MapPolicy for mapObsBin.");

Created default MapPolicy for mapObsBin.


#### Put an Observation Counter in the Map 

The experiment continued past the original end date. The new work requires keeping track of the total number of observations.

In [28]:
String mapkeyObsCount = "Count";
Integer numObservations = 3;

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record sizeOfRecordWithObsCounter = client.operate(null, key, 
    MapOperation.put(mapObsBinPolicy, mapObsBin, Value.get(mapkeyObsCount), Value.get(numObservations))
    );
Record recordWithObsCount = client.get(null, key);

System.out.println("The original Observations Map: " + record.getValue(mapObsBin));
System.out.println("The Observations Map with Counter: " + recordWithObsCount.getValue(mapObsBin));

The original Observations Map: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, 14567={lat=35, long=30}}
The Observations Map with Counter: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, Count=3, 14567={lat=35, long=30}}


#### Add a new Observation

For this tutorial, adding an observation does not require **strong consistency**. Aerospike Enterprise Edition supports strong consistency. 

For more information on [strong consistency](https://www.aerospike.com/docs/architecture/consistency.html), go [here](https://www.aerospike.com/docs/architecture/consistency.html). 

In [29]:
int newObsTimestamp = 15678;
int newObsLat = 80;
int newObsLong = 110;

HashMap <Integer, HashMap> mapNewObs = new HashMap <Integer, HashMap>();
HashMap <String, Integer> mapNewCoords = new HashMap <String, Integer>();
mapNewCoords.put("lat", newObsLat);
mapNewCoords.put("long", newObsLong);

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record sizeOfNewObs = client.operate(null, key, 
    MapOperation.put(mapObsBinPolicy, mapObsBin, Value.get(newObsTimestamp), Value.get(mapNewCoords))
    );
Record recordWithNewObs = client.get(null, key);

System.out.println("The Original Observation Map: " + record.getValue(mapObsBin));
System.out.println("The Size of the new Obs Map: " + sizeOfNewObs.getValue(mapObsBin));
System.out.println("The Map with New Observation: " + recordWithNewObs.getValue(mapObsBin));

The Original Observation Map: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, Count=3, 14567={lat=35, long=30}}
The Size of the new Obs Map: 5
The Map with New Observation: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, Count=3, 15678={lat=80, long=110}, 14567={lat=35, long=30}}


#### Remove the Oldest Observation by Index

This study only maintains the three most recent observations.

In [ ]:
Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record oldObs = client.operate(null, key, 
    MapOperation.removeByIndex(mapObsBin, firstIdx, MapReturnType.KEY_VALUE)
    );
Record updatedRecord = client.get(null, key);

System.out.println("The Original Observation Map: " + record.getValue(mapObsBin));
System.out.println("The Removed Observation: " + oldObs.getValue(mapObsBin));
System.out.println("The New Map: " + updatedRecord.getValue(mapObsBin));

#### Increment the Observation Counter

When incrementing a map value, Aerospike returns the new value as a convenience.

In [30]:
int incNum = 1;

Key key = new Key(mapNamespace, mapSet, theKey);
Record record = client.get(null, key);
Record obsCount = client.operate(null, key, 
    MapOperation.increment(mapObsBinPolicy, mapObsBin, Value.get(mapkeyObsCount), Value.get(incNum))
    );
Record updatedRecord = client.get(null, key);

System.out.println("The Original Observation Map: " + record.getValue(mapObsBin));
System.out.println("The New Count: " + obsCount.getValue(mapObsBin));
System.out.println("The New Map: " + updatedRecord.getValue(mapObsBin));

The Original Observation Map: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, Count=3, 15678={lat=80, long=110}, 14567={lat=35, long=30}}
The New Count: 4
The New Map: {13456={lat=-25, long=-50}, 12345={lat=-85, long=-130}, Count=4, 15678={lat=80, long=110}, 14567={lat=35, long=30}}


## DELETING the Records and Closing Server Connection

1. Use the **asinfo** administration tool to dump the set containing our list data.
2. Close the client's connection to the Aerospike cluster.

In [ ]:
%sh asinfo -v "truncate:namespace=test;set=mapset1;"
client.close();
System.out.println("Index dropped and server connection closed.");

## Aerospike Does Maps

Aerospike and its Java Client are up to the task of working with your map data. It provides rich operations to read and update list data using index, mapkey, value, and rank. Not modeled in this tutorial, Aerospike map operation also supports nested map, by assigning **CTX** or contexts to operations.

For more [information on contexts](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/CTX.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/CTX.html). For [examples of contexts](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html), go [here](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html). 

## What's Next?

Want to learn more?


### Next Steps

Have questions? Don't hesitate to reach out if you have additional questions about working with lists at https://discuss.aerospike.com/.

Want to check out other Java notebooks?
1. [Hello, World](hello_world.ipynb)
2. [Aerospike Query and UDF](query_udf.ipynb)
3. [Simple Put Get Example](SimplePutGetExample.ipynb)
4. [Working with Twitter Data](tweetaspike.ipynb)
5. [Working with Lists](java-working_with_lists.ipynb)

Are you running this from Binder? [Download the Aerospike Notebook Repo](https://github.com/aerospike-examples/interactive-notebooks) and work with Aerospike Database and Jupyter locally using a Docker container.

### Additional Resources

* What else can we do in Java? See the [Aerospike Java Client](https://github.com/aerospike/aerospike-client-java). 
* What other ways can we work with Maps? Take a look at [Aerospike's Map Operations](https://www.aerospike.com/apidocs/java/com/aerospike/client/cdt/MapOperation.html).
* What are Namespaces, Sets, and Bins? Check out the [Aerospike Data Model](https://www.aerospike.com/docs/architecture/data-model.html). 
* How robust is the Aerospike database? Browses the [Aerospike Database Architecture](https://www.aerospike.com/docs/architecture/index.html).